In [2]:
# !pip freeze

In [7]:
# !pip install torch

In [1]:
# !pip install pandas
# !pip install open_clip_torch
# !pip uninstall transformers --yes
# !pip install transformers

In [12]:
import pandas as pd
import torch
from PIL import Image
import open_clip
import transformers

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [4]:
df = pd.read_csv('image_new.csv')

In [6]:
del df['Unnamed: 0']

In [7]:
model, _, preprocess = open_clip.create_model_and_transforms('xlm-roberta-large-ViT-H-14', 'frozen_laion5b_s13b_b90k')

In [8]:
model.to(device)

CustomTextCLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-31): 32 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1280,), eps=1e-05,

In [9]:
df

,image,temp,label
0,gorey06ac982080e86f913e1ce19d00970315.jpg,gor,0
1,gorey0_Terrifier-2.jpg,gor,0
2,gorey1.jpg,gor,0
3,gorey1498729.jpg,gor,0
4,gorey1lqljQa.jpg,gor,0
...,...,...,...
10517,pos_p_sddefault.jpg,pos,1
10518,pos_p_sea-cliff-hotel-general-fc98067.jpg,pos,1
10519,pos_p_secret-romantic-date-night-kissing-coupl...,pos,1
10520,pos_p__bkuO7brYtpSveutq5DWqIL2qQJ1Adz6o_9rBeFT...,pos,1


In [19]:
def getImageFeature(image):
    with torch.no_grad(), torch.cuda.amp.autocast():
        image = preprocess(Image.open(image)).unsqueeze(0)
        image_features = model.encode_image(image.to(device))[0]
    return image_features.cpu().numpy()

In [20]:
getImageFeature("pos_2179_random human.jpg")

array([-0.606 ,  0.607 ,  0.5938, ..., -0.629 ,  0.0891,  0.1228],
      dtype=float16)

In [21]:
df['material_id'] = "image" + df.index.astype(str)

In [22]:
df.isna().sum()

image          0
temp           0
label          0
material_id    0
dtype: int64

In [24]:
cd imaged

/common/home/projectgrps/IS424/IS424G20/imaged


/common/home/projectgrps/IS424/IS424G20/jupyterlab-venv-pytorch-py311/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [32]:
cd imaged

/common/home/projectgrps/IS424/IS424G20/imaged


In [33]:
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['image'] 
        text_feature = getImageFeature(text)
        material_id = row['material_id']
        torch.save(text_feature, f'../image_embeddings/{material_id}.pt')
    except Exception as e:
        # Handle exceptions here if needed
        print(f"Error processing row {index}: {e}")

  5%|▌         | 536/10522 [00:17<05:20, 31.20it/s]/common/home/projectgrps/IS424/IS424G20/jupyterlab-venv-pytorch-py311/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 10522/10522 [04:41<00:00, 37.37it/s]


  adding: image_embeddings/ (stored 0%)
  adding: image_embeddings/image3150.pt (deflated 29%)
  adding: image_embeddings/image7627.pt (deflated 29%)
  adding: image_embeddings/image5096.pt (deflated 30%)
  adding: image_embeddings/image5823.pt (deflated 28%)
  adding: image_embeddings/image8021.pt (deflated 29%)
  adding: image_embeddings/image9057.pt (deflated 28%)
  adding: image_embeddings/image10252.pt (deflated 29%)
  adding: image_embeddings/image7011.pt (deflated 29%)
  adding: image_embeddings/image8269.pt (deflated 29%)
  adding: image_embeddings/image8856.pt (deflated 29%)
  adding: image_embeddings/image7810.pt (deflated 28%)
  adding: image_embeddings/image1716.pt (deflated 29%)
  adding: image_embeddings/image3646.pt (deflated 29%)
  adding: image_embeddings/image3805.pt (deflated 28%)
  adding: image_embeddings/image2659.pt (deflated 29%)
  adding: image_embeddings/image3616.pt (deflated 29%)
  adding: image_embeddings/image9270.pt (deflated 29%)
  adding: image_embeddin